In [ ]:
import numpy as np 
import pandas as pd 
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import randint
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd 
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from copy import deepcopy
from sklearn.feature_selection import f_classif
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.svm import LinearSVC
import pandas as pd
from scipy import stats
from statsmodels.stats import weightstats as stests
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [ ]:
train  = pd.read_csv('../input/students-performance-in-exams/StudentsPerformance.csv');

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
print("The number of traning examples(data points) = %i " % train.shape[0])
print("The number of features = %i " % train.shape[1])


In [ ]:
train.info()


In [ ]:
train.describe()



In [ ]:
train.isnull()

**NO Null Values!
**

In [ ]:
train.isnull().sum()

In [ ]:
train['lunch'].isnull().sum()
 

In [ ]:
duplicate = train[train.duplicated()]
duplicate.size


**NO Duplicates!******

In [ ]:
train['total percentage'] = ((train['math score']+train['reading score']+train['writing score'])/3).round(decimals=2)


In [ ]:
train.head()


In [ ]:
train['total percentage'].value_counts(bins=2, sort=False)

In [ ]:
bins = [0,50,100]
names = ['Failed','Passed']
train['status'] =  pd.cut(train['total percentage'], bins, labels=names)
train

In [ ]:
NumberedGender = {'female':0,'male':1}
train['gender']=train['gender'].map(NumberedGender)
train['gender'] = pd.to_numeric(train['gender'])
train.head()

In [ ]:
train['race/ethnicity'].unique()

In [ ]:
NumberedRace = {'group A':0,'group B':1,'group C':2,'group D':3,'group E':4}
train['race/ethnicity']=train['race/ethnicity'].map(NumberedRace)
train['race/ethnicity'] = pd.to_numeric(train['race/ethnicity'])
train.head()

In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(1, 4, 1)
fig = train.boxplot(column='math score')
fig.set_title('')
fig.set_ylabel('math score')
 
plt.subplot(1, 4, 2)
fig = train.boxplot(column='reading score')
fig.set_title('')
fig.set_ylabel('reading score')

plt.subplot(1, 4, 3)
fig = train.boxplot(column='writing score')
fig.set_title('')
fig.set_ylabel('writing score')

plt.subplot(1, 4, 4)
fig = train.boxplot(column='total percentage')
fig.set_title('')
fig.set_ylabel('total percentage')

In [ ]:

g = sns.displot(train["math score"], color="c", label="Skewness : %.2f"%(train["math score"].skew()))
g = sns.displot(train["reading score"], color="y", label="Skewness : %.2f"%(train["reading score"].skew()))
g = sns.displot(train["writing score"], color="m", label="Skewness : %.2f"%(train["writing score"].skew()))
g = sns.displot(train["total percentage"], color="g", label="Skewness : %.2f"%(train["total percentage"].skew()))



In [ ]:
train.head()

In [ ]:
g = sns.displot(train["math score"], color="c", label="Skewness : %.2f"%(train["math score"].skew()))
g = sns.displot(train["reading score"], color="y", label="Skewness : %.2f"%(train["reading score"].skew()))
g = sns.displot(train["writing score"], color="m", label="Skewness : %.2f"%(train["writing score"].skew()))
g = sns.displot(train["total percentage"], color="g", label="Skewness : %.2f"%(train["total percentage"].skew()))


In [ ]:
train.corr()

In [ ]:
corr = train.corr()
f, ax = plt.subplots(figsize=(20, 7))
cmap = sns.diverging_palette(123, 10, as_cmap=True)
sns.heatmap(corr,linewidths=.5, annot= True)

In [ ]:
sns.pairplot(data=train[['gender','race/ethnicity','total percentage']],
             hue="total percentage", dropna=True)

In [ ]:
cleanup_nums = { "status": {"Failed": 0, "Passed": 1} }
train.replace(cleanup_nums, inplace=True)
train.head()

In [ ]:
target = train['status']

In [ ]:
data = train[['gender','race/ethnicity']]

Hypothesis : There is no difference in the pass/fail status between students females and males.

In [ ]:
import random

male = train[train['gender'] == 1]
female = train[train['gender'] == 0]

## empty list for storing mean sample
m_mean_samples = []
f_mean_samples = []

for i in range(50):
    m_mean_samples.append(np.mean(random.sample(list(male['status']),50,)))
    f_mean_samples.append(np.mean(random.sample(list(female['status']),50,)))
    
ttest,pval = stats.ttest_rel(m_mean_samples, f_mean_samples)
print('T-test: ',float(pval))
if pval<0.05:
    print("reject null hypothesis")
else:
    print("accept null hypothesis")
ztest ,pval1 = stests.ztest(m_mean_samples,f_mean_samples, value=0,alternative='two-sided')
print('Z-test: ',float(pval1))
if pval1<0.05:
    print("reject null hypothesis")
else:
    print("accept null hypothesis")

In [ ]:
sns.barplot(x=train['gender'],y=train['status']);


Hypothesis : There is no difference in the pass/fail status between students Race/Ethnicity


In [ ]:
GroupA = train[train['race/ethnicity']==0]
GroupB = train[train['race/ethnicity']==1]
GroupC = train[train['race/ethnicity']==2]
GroupD = train[train['race/ethnicity']==3]
GroupE = train[train['race/ethnicity']==4]
gA_mean_samples=[]
gB_mean_samples=[]
gC_mean_samples=[]
gD_mean_samples=[]
gE_mean_samples=[]
for i in range(50):
    gA_mean_samples.append(np.mean(random.sample(list(GroupA['status']),50,)))
    gB_mean_samples.append(np.mean(random.sample(list(GroupB['status']),50,)))
    gC_mean_samples.append(np.mean(random.sample(list(GroupC['status']),50,)))
    gD_mean_samples.append(np.mean(random.sample(list(GroupD['status']),50,)))
    gE_mean_samples.append(np.mean(random.sample(list(GroupE['status']),50,)))    
F, p = stats.f_oneway(gA_mean_samples, gB_mean_samples, gC_mean_samples,gD_mean_samples,gE_mean_samples)
print("p-value for significance is: ", p)
if p<0.05:
    print("reject null hypothesis")
else:
    print("accept null hypothesis")


In [ ]:
sns.barplot(x=train['race/ethnicity'],y=train['status']);

In [ ]:
sns.barplot(x=train['race/ethnicity'],y=train['status'],hue=train['gender']);